---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

#df = pd.DataFrame(doc)
df=pd.Series(doc)
#df.rename(columns={0:'record'},inplace=True)
#str1=df.loc[330,'record']
#print(str1)

In [2]:
import numpy as np
def date_sorter():
    
    b1_1 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    b1_2 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    b1 = pd.concat([b1_1,b1_2])
    b1.reset_index(inplace=True)
    b1_index = b1['level_0']
    
    b2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    b2.reset_index(inplace=True)
    b2_index = b2['level_0']
    
    b3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    b3.reset_index(inplace=True)
    b3_index = b3['level_0']
    
    b6 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    b6.reset_index(inplace=True)
    b6_index = b6['level_0']
    record=[]
    for i in b6_index:
        if not(i in b1_index.values):
            record.append(i)
    record = np.asarray(record)
    b6 = b6[b6['level_0'].isin(record)]

    
    b7_1= df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    b7_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    b7 = pd.concat([b7_1,b7_2])
    b7.reset_index(inplace=True)

    b7_index = b7['level_0']
    record=[]
    for i in b7_index:
        if not((i in b2_index.values) | (i in b3_index.values) | (i in b6_index.values)):
            record.append(i)
    record = np.asarray(record)
    b7 = b7[b7['level_0'].isin(record)]
    
    s = b1.level_0.values.tolist()+b2.level_0.values.tolist()+b3.level_0.values.tolist()+b6.level_0.values.tolist()+b7.level_0.values.tolist()
    s = np.asarray(s)
    
    b1.columns=['level_0','match','month','day','year']
    b1['year']=b1['year'].apply(str)
    b1['year']=b1['year'].apply(lambda x: '19'+x if len(x)<=2 else x)
   
    b2[1] = b2[1].apply(lambda x: x.replace(',',''))
    b2['day'] = b2[1].apply(lambda x:x.split(' ')[0])
    b2['year'] = b2[1].apply(lambda x:x.split(' ')[1])
    b2.columns=['level_0','match','month','day-year','day','year']
    b2.drop('day-year',axis=1,inplace=True) 
    
    b3.columns=['level_0','match','day','month','year']
    b3['day'] = b3['day'].replace(np.nan,-99)
    b3['day'] = b3['day'].apply(lambda x: 1 if int(x)==-99 else x)

    b3['month'] = b3.month.apply(lambda x: x[:3])
    b3['month'] = pd.to_datetime(b3.month, format='%b').dt.month
    
    b6.columns=['level_0','match','month','year']
    b6['day']=1
  
    b7.columns=['level_0','match','year']
    b7['day']=1
    b7['month']=1
   
    final = pd.concat([b1,b2,b3,b6,b7])
    final['date'] =pd.to_datetime(final['month'].apply(str)+'/'+final['day'].apply(str)+'/'+final['year'].apply(str))
    final = final.sort_values(by='level_0').set_index('level_0')

    myList = final['date']
    answer = pd.Series([i[0] for i in sorted(enumerate(myList), key=lambda x:x[1])],np.arange(500))
    return answer

date_sorter()
    

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

AttributeError: 'DataFrame' object has no attribute 'str'

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

In [ ]:
def date_sorter_1():
    df['new']=df['record'].str.extract(r'((\d{1,2}(?:/|-| )\d{1,2}(?:/|-| )\d{2,4})|((?:\d{1,2}(?:/|-| |,|.))?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]+(?:/|-| |,|.)\d{1,2}?[sth]{,2}(?:/|-| |,|.){,2}\d{2,4})|(\d{1,2}/\d{4})|(?:\d{4}))')
    #df['new']=df['record'].str.extract(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b|(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b|((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})|((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})|(\d{1,2})[/](\d{4})|[a-z]?[^0-9](\d{4})[^0-9]|^(\d{4})[^0-9]')
    str1='/01/'
    str2='01/01/'
    df['new2']=df['new'].str.replace(r'/|-|,| |\.','/')
    def gf(x):
        x=str(x)
        if  x.count(r'/')==1:
            if x.find(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)')==-1:
                return x.split('/')[0]+str1+x.split('/')[1]
            else:
                print ('01/'+x )
                return '01/'+x         
                
        elif x.count(r'/')==0:
            return str2+str(x)
        else:
            return x
    df['new3']=df['new2'].apply(gf)
    df.loc[271,'new']=df.loc[271,'new'].replace(r'7787','2008')
    df.loc[392,'new']=df.loc[392,'new'].replace(r'67','01')

    #return  df[df['m']>'12']
    df['new_time']=pd.to_datetime(df['new3'],errors='ignore')#df['y'].apply(str)+'/'+df['m'].apply(str)+'/'+df['d'].apply(str))
    df['rank']=df.index
    df.sort_values(by='new_time',inplace=True)
    df.reset_index(drop=True)
 
    return df